# CNN
Using pytorch nn, the code will build cnn from scratch
## Dataset
data set will be black and white picture (inchannel = 1), where capture basic places with labels. The size of the data is 64 x 64.
##  nn liabraries that are going to be used in the project
### nn.Module
This is the basic class for all neaural network modules. all neaural network built by  nn should be subcalss of this class
### nn.Conv2
2d convolutional process where the params are (in_channel, out_channel, kernel_size, ...). Since our image is 2d we will do 2d convolution.
### nn.MaxPool2d
2d Maxpooling. from 2d kernel, pick max value on each kernel.
### nn.ReLU
Applies recifiedlinear unit function. If the input is negative, it is zero, otherwise the output will be input.
### nn.Linear
Applies linear transformation (y = xA**T + b) on input size and output size.

In [5]:
# import necessary liabraries
from torch import nn

## Architectural Design
1. (2d convolutional -> max pooling with 3x3 kenel -> relu) 
2. (2d convolution -> max pooling with 3x3 kernel -> relu)
3. two fully connected layers

### Loss Criterion
To calculate the difference, it used 'mean' reduction in the loss criterion by using method of cross entropy

In [4]:
class CNN1(nn.Module):
    def __init__(self):
        """
        Init function to define the layers and loss function

        Note:Read Pytorch documention to understand what it means

        """
        super().__init__()

        self.conv_layers = nn.Sequential()
        # input: 1x(64x64)
        self.conv_layers.add_module("conv_1", nn.Conv2d(1, 10, kernel_size=5))
        # output/input: 10 x (60x60)
        self.conv_layers.add_module("maxpool1", nn.MaxPool2d(kernel_size=3))
        self.conv_layers.add_module("relu_1", nn.ReLU())
        # output: 10 x (20 x 20)

        # input: 10 x (20 x 20)
        self.conv_layers.add_module("conv_2", nn.Conv2d(10, 20, kernel_size=5))
        # output/input: 20 x (16x16)
        self.conv_layers.add_module("maxpool2", nn.MaxPool2d(kernel_size=3))
        self.conv_layers.add_module("relu_2",  nn.ReLU())
        # output: 20 x (5 x 5)

        self.conv_layers.add_module("flatten",  nn.Flatten())
        self.conv_layers.add_module("relu_3",  nn.ReLU())
        # output: 20 x 5 x 5 = 500

        # this is fully connected layers
        self.fc_layers = nn.Sequential()
        self.fc_layers.add_module("linear_1", nn.Linear(in_features=500, out_features=100))
        self.fc_layers.add_module("linear_2", nn.Linear(in_features=100, out_features=15))

        self.loss_criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        """
        Args:
        -   x: the input image [Dim: (N,C,H,W)] : data type = Tensor
        Returns:
        -   y: the output (raw scores) of the net [Dim: (N,15)] : data type = Tensor
        """
        return self.fc_layers(self.conv_layers(x))

## BatchNorm2d
Applied 4d Batch normalization. The mean and standard-deviation are calculated per-dimension over the mini-batches of 2D inputs with additional channel dimension and γ\gammaγ and β\betaβ are learnable parameter vectors of size of the input.
funtion is described on the website: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html

In [ ]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()

        self.fc_layers = nn.Sequential()

        self.conv_layers = nn.Sequential()
        self.conv_layers.add_module("conv_1", torch.nn.Conv2d(1, 10, kernel_size=5))
        self.conv_layers.add_module("BN1", nn.BatchNorm2d(num_features=10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.conv_layers.add_module("maxpool1", torch.nn.MaxPool2d(kernel_size=3, padding=1, stride = 2))
        self.conv_layers.add_module("relu_1", torch.nn.ReLU())
        
        self.conv_layers.add_module("conv_2", torch.nn.Conv2d(10, 20, kernel_size=5))
        self.conv_layers.add_module("BN2", nn.BatchNorm2d(num_features=20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.conv_layers.add_module("maxpool2", torch.nn.MaxPool2d(kernel_size=3, padding=1, stride= 2))
        self.conv_layers.add_module("relu_2", torch.nn.ReLU())
        
        self.conv_layers.add_module("dropout", torch.nn.Dropout())
        self.conv_layers.add_module("conv_3", torch.nn.Conv2d(20, 50, kernel_size=5))
        self.conv_layers.add_module("maxpool3", torch.nn.MaxPool2d(kernel_size=3, padding=1, stride= 2))
        self.conv_layers.add_module("relu_3", torch.nn.ReLU())
        self.conv_layers.add_module("flatten", torch.nn.Flatten())


        self.fc_layers.add_module("linear_1", nn.Linear(in_features=1250, out_features=500))
        self.fc_layers.add_module("linear_2", nn.Linear(in_features=500, out_features=15))

        self.loss_criterion = nn.CrossEntropyLoss()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Perform the forward pass with the net

        Args:
        -   x: the input image [Dim: (N,C,H,W)]
        Returns:
        -   y: the output (raw scores) of the net [Dim: (N,15)]
        """
        
        conv_output = self.conv_layers(x)
        model_output = self.fc_layers(conv_output)

        return model_output
